### Pythonic way of extracting numbers from John Hopkins Covid19 Dashboard using ArcGIS API for Python


*As we all know ArcGIS Python API has rich pythonic library with all built-in functions. 
With the introduction of Spatial DataFrame into ArcGIS API, it enables us to immediately convert feature layers into dataframe which can be analyzed further. This is an attempt to automatically extract Country wise numbers (Active, Deaths, Confirmed) and US-County-Wise numbers (Confirmed,Deaths)*

### Reference Links
* [ArcGIS-Python-API](https://developers.arcgis.com/python/guide/)
* [Pandas-Styling](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html)
* [Matplotlib-colormaps](https://matplotlib.org/examples/color/colormaps_reference.html)

In [3]:
from arcgis.gis import GIS
import pandas as pd
import numpy as np
%matplotlib inline

#This is for displaying all outputs interactively in the notebook, not just the last-one which is default in Jupyter.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

'''For reference, the options for that are 'all', 'none', 'last' and 'last_expr'. 
The difference between 'last' and 'last_expr': if your cell ends with, say, a loop containing an expression, 
'last' will show you the results from that expression in each iteration of the loop. 
'last_expr' (the default) won't show that: it will only display the result of a bare expression at the end of the cell.'''
#https://stackoverflow.com/questions/36786722/how-to-display-full-output-in-jupyter-not-only-last-result


gis = GIS()

#Get the item ids of Layers from ArcGIS Online
itemUSCounties = gis.content.get('628578697fb24d8ea4c32fa0c5ae1843')
itemCountries = gis.content.get('c0b356e20b30490c8b8b4c7bb9554e7c')

#Get the feature layers from item.
deathsByUSCounty = itemUSCounties.layers[0]
deathsByCountry = itemCountries.layers[2]

#Create a country wise deaths spatial dataframe and check no of rows & columns
sdf = pd.DataFrame.spatial.from_layer(deathsByCountry)
print(f"Countrywise Deaths Feature Layer has {sdf.shape[0]} rows and {sdf.shape[1]} columns")

#Create a US County wise deaths spatial dataframe and check no.of rows & columns
sdfc = pd.DataFrame.spatial.from_layer(deathsByUSCounty)
print(f"US Counties Feature Layer has {sdfc.shape[0]} rows and {sdfc.shape[1]} columns")

#Adjust as per your requirements to display max rows/columns
pd.set_option('display.max_rows',100000)
pd.set_option('display.max_columns',10)

#Get Country wise Deaths,confirmed cases grouped by country sorted by deaths
sdfd = sdf[['Country_Region','Deaths','Confirmed','Recovered','Active']]
deathcount = sdfd.sort_values(by='Deaths',ascending=False)

deathcount.reset_index(drop=True,inplace=True)

#Apply gradient style and hide index
CountryWiseDeaths = deathcount.style.format({"Country_Region": lambda x:x.upper()})\
.background_gradient(cmap="Blues").hide_index()

#Apply bar style and hide index
CountryWiseDeaths_bar = deathcount.style.format({"Country_Region": lambda x:x.upper()})\
.bar(subset=["Deaths",], color='lightgreen')\
.bar(subset=["Confirmed"], color='#ee1f5f')\
.bar(subset=["Recovered"], color='#FFA07A')\
.bar(subset=["Active"],color="#ffa600").hide_index()

def plot_pandas_style(styler):
    """Currently pandas stylers have an unforatinate indent in the html they render
    See: https://github.com/pandas-dev/pandas/issues/15651

    Example
    -------
    plot_pandas_style(df.style.set_caption('this is my caption'))
    """
    from IPython.core.display import HTML
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)

#Display both
CountryWiseDeaths_bar
CountryWiseDeaths

#export to excel, csv as per your needs.
CountryWiseDeaths.to_excel('Covid-CountryWiseDeaths-data.xlsx')


"For reference, the options for that are 'all', 'none', 'last' and 'last_expr'. \nThe difference between 'last' and 'last_expr': if your cell ends with, say, a loop containing an expression, \n'last' will show you the results from that expression in each iteration of the loop. \n'last_expr' (the default) won't show that: it will only display the result of a bare expression at the end of the cell."

Countrywise Deaths Feature Layer has 177 rows and 10 columns
US Counties Feature Layer has 3173 rows and 16 columns


Country_Region,Deaths,Confirmed,Recovered,Active
ITALY,10779,97689,13030,73880
SPAIN,6606,78799,14709,57484
CHINA,3304,82122,75582,3236
IRAN,2640,38309,12391,23278
FRANCE,2609,40704,7226,30869
US,2409,136880,2612,0
UNITED KINGDOM,1231,19772,151,18390
NETHERLANDS,772,10930,253,9905
GERMANY,490,61164,9211,51463
BELGIUM,431,10836,1359,9046


Country_Region,Deaths,Confirmed,Recovered,Active
ITALY,10779,97689,13030,73880
SPAIN,6606,78799,14709,57484
CHINA,3304,82122,75582,3236
IRAN,2640,38309,12391,23278
FRANCE,2609,40704,7226,30869
US,2409,136880,2612,0
UNITED KINGDOM,1231,19772,151,18390
NETHERLANDS,772,10930,253,9905
GERMANY,490,61164,9211,51463
BELGIUM,431,10836,1359,9046


In [5]:
#Renaming 'Admin2' column in US Counties dataset to 'County' in the dataset
sdfc.rename(columns={"Admin2":"County"},inplace=True)

#Extract the required columns and sort them by deaths
sdfcounty = sdfc[['Province_State','County','Confirmed','Deaths']].copy()
sdfcounty.sort_values(by=['Deaths'],ascending=[False],inplace=True)

#Creating a pivot table and summing aggregation
df_pivot = pd.pivot_table(sdfcounty,values=['Confirmed','Deaths'],index=['Province_State','County'],aggfunc=np.sum)
df_pivot.sort_values(by='Deaths',ascending=False,inplace=True)

#Eliminating counties that have both confirmed and death cases as 0
#We want counties where there are confirmed cases but not deaths hence doing 'or' condition
filt = ~(df_pivot.Deaths == 0) | ~(df_pivot.Confirmed == 0)
df_pivot_filt = df_pivot[filt]


#Get the aggregated state,county dataframe without filtering
#----Use this if for exporting if you dont want filtering---------
b = pd.concat([
    d.append(d.sum().rename((k, 'TOTAL')))
    for k, d in df_pivot.groupby(level=0)
]).append(df_pivot.sum().rename(('USA-TOTAL', '')))

#b.style.apply(lambda x: ['background-color:red' if (x.Deaths>0) else '' for i in x], axis=1)

#Get the aggregated state, county dataframe with filtering along with subtotals per state & USA-Total.
a = pd.concat([
     d.append(d.sum().rename((k, 'TOTAL')))
     for k, d in df_pivot_filt.groupby(level=0)
 ]).append(df_pivot_filt.sum().rename(('USA-TOTAL', '')))

def plot_pandas_style(styler):
    """Currently pandas stylers have an unforatinate indent in the html they render
    See: https://github.com/pandas-dev/pandas/issues/15651

    Example
    -------
    plot_pandas_style(df.style.set_caption('this is my caption'))
    """
    from IPython.core.display import HTML
    html = '\n'.join([line.lstrip() for line in styler.render().split('\n')])
    return HTML(html)


#Applying styles - Light Red for sub-totals, Light blue for deaths>0, wheat for grand-total
final_df = a.style.apply(lambda x:['background-color:#FA8072' if x.name[1]=='TOTAL' else'' for i in x],axis=1)\
.apply(lambda x:['background-color:#87CEFA' if x.Deaths>0 and x.name[1]!='TOTAL' else'' for i in x],axis=1)\
.apply(lambda x:['background-color:#F5DEB3' if x.name[0]=='USA-TOTAL' else'' for i in x],axis=1)

#Similar to what we did earlier, you can also apply bar-styles if needed

#Printing just for verification
final_df


#export to excel, csv as per your needs.
final_df.to_excel('Covid-US-Countywise-data.xlsx')